In [1]:
#Check Python Version
import shutil
import sys
sys.version

'3.8.2 (default, Jan 31 2023, 18:34:03) \n[GCC 12.2.0]'

In [2]:
#Upgrade pip
%pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [3]:
#Check TF & TFX Versioning
import tensorflow as tf
print(tf.__version__)
from tfx import v1 as tfx
print(tfx.__version__)

2023-02-21 14:53:48.860013: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-21 14:53:49.314521: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-21 14:53:49.314598: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-21 14:53:50.921443: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

2.11.0
1.12.0


In [4]:
#Setup Variables as examplegen_playground
import os

# Pipeline name
PIPELINE_NAME = "copy_tfrecords"

# Output directory to store artifacts generated from the pipeline.
PIPELINE_ROOT = './artifacts'
# Path to a SQLite DB file to use as an MLMD storage.
METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')
# Output directory where created models from the pipeline will be exported.
SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)

# Folder path to data
DATA_ROOT = './data/'

# Folder path to tfrecords
TFRECORDS_TRAIN_DATA_PATH = '../../../../Documents/tfrecord1_split_train/data_tfrecord-00000-of-00001.gz'
TFRECORDS_EVAL_DATA_PATH = '../../../../Documents/tfrecord2_split_eval/data_tfrecord-00000-of-00001.gz'

from absl import logging
logging.set_verbosity(logging.INFO)  # Set default logging level.

In [7]:
import json
from typing import Any, Dict, List

from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.v1.types.standard_artifacts import Examples

context = InteractiveContext()

# User input dictionary with split-names and their resepctive uri to tfrecords
tfrecords_dict={
    "Split-train":TFRECORDS_TRAIN_DATA_PATH,
    "Split-eval":TFRECORDS_EVAL_DATA_PATH
}

# Destination directory for source
destination_examples_artifact_uri = './examples/1/'

# Create a folder in destination_examples_artifact_uri for every key name
for key in tfrecords_dict:
    path = os.path.join(destination_examples_artifact_uri, key)
    if not os.path.exists(path):
        os.mkdir(path)
    # print(tfrecords_dict[key])
    shutil.copy(tfrecords_dict[key], path)

# CopyExampleGen execution process to replace Importer Node functionality
examples_importer = tfx.dsl.Importer(
    source_uri=destination_examples_artifact_uri,
    artifact_type=tfx.types.standard_artifacts.Examples).with_id(
        'examples_importer')

context.run(examples_importer)

# statistics_gen = tfx.components.StatisticsGen(
#     examples=examples_importer.outputs['result'])

# context.run(statistics_gen)

INFO:absl:Running driver for examples_importer
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Processing source uri: ./examples/1/, properties: {}, custom_properties: {}
INFO:absl:Running executor for examples_importer
INFO:absl:Running publisher for examples_importer
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: examples_importer
    execution_id: 1
    outputs:
        result: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 1
        type_id: 14
        uri: "./examples/1/"
        custom_properties {
          key: "is_external"
          value {
            int_value: 1
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.12.0"
          }
        }
        , artifact_type: id: 14
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        base_type: DATASET
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [ ]:
context = InteractiveContext()

# Destination directory for source
destination_examples_artifact_uri = './examples/'

# Importer Node accepting directory structure of examples/1/Split-eval/tfrecords.gz and examples/1/Split-train/tfrecords.gz
examples_importer = tfx.dsl.Importer(
    source_uri=destination_examples_artifact_uri,
    artifact_type=tfx.types.standard_artifacts.).with_id(
        'examples_importer')

context.run(examples_importer)

In [ ]:
context = InteractiveContext()

# Importer Node
examples_importer = tfx.dsl.Importer(
    source_uri=TFRECORDS_DATA_PATH,
    artifact_type=tfx.types.standard_artifacts.Examples).with_id(
        'examples_importer')

print("Output (examples_importer.outputs['result'].get()[0].version): ", examples_importer.outputs['result'].get()[0])

context.run(examples_importer)

In [ ]:
from tfx.components.example_gen.import_example_gen.component import ImportExampleGen
from standard_artifacts_v2 import ExampleTfrecords

context = InteractiveContext()

examples='./external_tfrecords/tfrecords/Split_train/'
example_gen = ImportExampleGen(input_base=examples)

context.run(example_gen)

print("Output (example_gen.outputs['examples'].get()[0].version): ", example_gen.outputs['examples'].get()[0])

# examples_importer = tfx.dsl.Importer(
#     source_uri=TFRECORDS_DATA_PATH,
#     artifact_type=tfx.types.standard_artifacts.Examples).with_id(
#         'examples_importer')

# examples_tfrecords = ExampleTfrecords(

# )

# statistics_gen = tfx.components.StatisticsGen(
#     examples=example_gen.outputs['examples'])

# context.run(statistics_gen)


In [ ]:
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

context = InteractiveContext()

# _data_filepath = os.path.join(DATA_ROOT, "data.csv")

# print(data_filepath)
example_gen = tfx.components.CsvExampleGen(input_base=DATA_ROOT)

context.run(example_gen)

In [ ]:
import os
import absl
import json
import pprint
import tempfile

from typing import Any, Dict, List, Text

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import apache_beam as beam

from absl import logging

# from tfx.components.example_gen.base_example_gen_executor import BaseExampleGenExecutor
# from base_example_gen_executor_v2.py import BaseExampleGenExecutorV2
from tfx.components.example_gen.component import FileBasedExampleGen
from tfx.components.example_gen import utils
from tfx.dsl.components.base import executor_spec

from tfx.types import artifact
from tfx.types import artifact_utils
from tfx.types import channel
from tfx.types import standard_artifacts
from tfx.types.standard_artifacts import Examples

from tfx.dsl.component.experimental.annotations import InputArtifact
from tfx.dsl.component.experimental.annotations import OutputArtifact
from tfx.dsl.component.experimental.annotations import Parameter
from tfx.dsl.component.experimental.decorators import component
from tfx.types.experimental.simple_artifacts import Dataset

from tfx import v1 as tfx
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

builder = tfds.builder('mnist')
# 1. Create the tfrecord files (no-op if already exists)
builder.download_and_prepare()
# 2. Load the `tf.data.Dataset`
ds = builder.as_dataset(split='train', shuffle_files=False)
print(ds)



In [ ]:
###################################################################################
### [USE CASE 1]
### Tfrecords are already created but not in Examples Artifact form

### This component will:
### 1. Accept a dict with with {'split_name': './path/to/split_name/tfrecords.gz'}
### 2. Add them to a folder to follow Example Artifact directory structure
### 3. Import them with Importer node to register the external resource into MLMD
###################################################################################
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

# Create pipeline to run Importer node
def _create_pipeline(
  pipeline_name: str,
  pipeline_root: str,
  data_root: str,
  metadata_path: str
  ) -> tfx.dsl.Pipeline:

  context = InteractiveContext()
  ### Get external source data
  # Source directory of external Examples Artifacts
  source_examples_artifact_uri = '../artifacts/'

  # Destination directory for source
  destination_examples_artifact_uri = './artifacts/'

  # Get all files from source_examples_artifact_uri
  files = os.listdir(source_examples_artifact_uri)

  # Import source files to destination
  shutil.copytree(source_examples_artifact_uri, destination_examples_artifact_uri, dirs_exist_ok=True)


  # example_gen = tfx.components.CsvExampleGen(input_base=data_root)
  # print(example_gen.outputs['examples'])

  # Importer Node
  examples_importer = tfx.dsl.Importer(
      source_uri=TFRECORDS_DATA_PATH,
      artifact_type=tfx.types.standard_artifacts.Examples).with_id(
          'examples_importer')

  context.run(examples_importer)
  print("IMPORTER_NODE: ", examples_importer.outputs)
  print("IMPORTER_NODE: ", examples_importer.outputs['result'].additional_properties.keys)
  print(list(examples_importer.outputs['result'].additional_properties.keys()))


  # Test if downstream component accepts output from Importer Node
  statistics_gen = tfx.components.StatisticsGen(
      examples=examples_importer.outputs['result'])


  # EXAMPLEGEN COMPONENT OUTPUT
  # Brings data into the pipeline.
  # example_gen = tfx.components.CsvExampleGen(input_base=data_root)
  # print("EXAMPLE_GEN: ", example_gen.outputs['examples'])

  # print("tfx.components.CsvExampleGen.outputs['examples'] output: \n\n",
  #   "Artifact Type: ", example_gen.outputs['examples']._artifact_type, "\n"
  #   "Producer Component Id: ", example_gen.outputs['examples'].producer_component_id, "\n"
  #   "Output Key: ", example_gen.outputs['examples'].output_key, "\n"
  #   "Additional Properties: ", example_gen.outputs['examples'].additional_properties, "\n"
  #   "Additional Custom Properties: ", example_gen.outputs['examples'].additional_custom_properties, "\n"
  # )
  # print(list(example_gen.outputs['examples'].additional_properties.keys()))

  components = [
    # example_gen,
    examples_importer,
    statistics_gen
  ]

  return tfx.dsl.Pipeline(
    pipeline_name=PIPELINE_NAME,
    pipeline_root=PIPELINE_ROOT,
    metadata_connection_config=tfx.orchestration.metadata
      .sqlite_metadata_connection_config(metadata_path),
    components=components
)

In [ ]:
tfx.orchestration.LocalDagRunner().run(
  _create_pipeline(
      pipeline_name=PIPELINE_NAME,
      pipeline_root=PIPELINE_ROOT,
      data_root=DATA_ROOT,
      metadata_path=METADATA_PATH)
  )

In [ ]:
import re

dict_output = {
	"name": "ValueError",
	"message": "No splits for examples artifact: Artifact(artifact: id: 1\ntype_id: 14\nuri: \"./examples_directory_build/\"\ncustom_properties {\n  key: \"is_external\"\n  value {\n    int_value: 1\n  }\n}\ncustom_properties {\n  key: \"tfx_version\"\n  value {\n    string_value: \"1.12.0\"\n  }\n}\n, artifact_type: id: 14\nname: \"Examples\"\nproperties {\n  key: \"span\"\n  value: INT\n}\nproperties {\n  key: \"split_names\"\n  value: STRING\n}\nproperties {\n  key: \"version\"\n  value: INT\n}\nbase_type: DATASET\n)",
	"stack": "\u001b[0;31m---------------------------------------------------------------------------\u001b[0m\n\u001b[0;31mValueError\u001b[0m                                Traceback (most recent call last)\n\u001b[0;32m/tmp/ipykernel_2659444/807081948.py\u001b[0m in \u001b[0;36m<cell line: 40>\u001b[0;34m()\u001b[0m\n\u001b[1;32m     38\u001b[0m     examples=examples_importer.outputs['result'])\n\u001b[1;32m     39\u001b[0m \u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0;32m---> 40\u001b[0;31m \u001b[0mcontext\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mrun\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0mstatistics_gen\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0m\n\u001b[0;32m~/.pyenv/versions/3.8.2/envs/tfx-3.8.2/lib/python3.8/site-packages/tfx/orchestration/experimental/interactive/notebook_utils.py\u001b[0m in \u001b[0;36mrun_if_ipython\u001b[0;34m(*args, **kwargs)\u001b[0m\n\u001b[1;32m     29\u001b[0m       \u001b[0;31m# __IPYTHON__ variable is set by IPython, see\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m     30\u001b[0m       \u001b[0;31m# https://ipython.org/ipython-doc/rel-0.10.2/html/interactive/reference.html#embedding-ipython.\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0;32m---> 31\u001b[0;31m       \u001b[0;32mreturn\u001b[0m \u001b[0mfn\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0;34m*\u001b[0m\u001b[0margs\u001b[0m\u001b[0;34m,\u001b[0m \u001b[0;34m**\u001b[0m\u001b[0mkwargs\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0m\u001b[1;32m     32\u001b[0m     \u001b[0;32melse\u001b[0m\u001b[0;34m:\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m     33\u001b[0m       logging.warning(\n\n\u001b[0;32m~/.pyenv/versions/3.8.2/envs/tfx-3.8.2/lib/python3.8/site-packages/tfx/orchestration/experimental/interactive/interactive_context.py\u001b[0m in \u001b[0;36mrun\u001b[0;34m(self, component, enable_cache, beam_pipeline_args)\u001b[0m\n\u001b[1;32m    162\u001b[0m         \u001b[0mtelemetry_utils\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mLABEL_TFX_RUNNER\u001b[0m\u001b[0;34m:\u001b[0m \u001b[0mrunner_label\u001b[0m\u001b[0;34m,\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m    163\u001b[0m     }):\n\u001b[0;32m--> 164\u001b[0;31m       \u001b[0mexecution_id\u001b[0m \u001b[0;34m=\u001b[0m \u001b[0mlauncher\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mlaunch\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mexecution_id\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0m\u001b[1;32m    165\u001b[0m \u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m    166\u001b[0m     return execution_result.ExecutionResult(\n\n\u001b[0;32m~/.pyenv/versions/3.8.2/envs/tfx-3.8.2/lib/python3.8/site-packages/tfx/orchestration/launcher/base_component_launcher.py\u001b[0m in \u001b[0;36mlaunch\u001b[0;34m(self)\u001b[0m\n\u001b[1;32m    204\u001b[0m       \u001b[0;31m# be immutable in this context.\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m    205\u001b[0m       \u001b[0;31m# output_dict can still be changed, specifically properties.\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0;32m--> 206\u001b[0;31m       self._run_executor(execution_decision.execution_id,\n\u001b[0m\u001b[1;32m    207\u001b[0m                          \u001b[0mcopy\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mdeepcopy\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0mexecution_decision\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0minput_dict\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m,\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m    208\u001b[0m                          \u001b[0mexecution_decision\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0moutput_dict\u001b[0m\u001b[0;34m,\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\n\u001b[0;32m~/.pyenv/versions/3.8.2/envs/tfx-3.8.2/lib/python3.8/site-packages/tfx/orchestration/launcher/in_process_component_launcher.py\u001b[0m in \u001b[0;36m_run_executor\u001b[0;34m(self, execution_id, input_dict, output_dict, exec_properties)\u001b[0m\n\u001b[1;32m     71\u001b[0m     \u001b[0;31m# be immutable in this context.\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m     72\u001b[0m     \u001b[0;31m# output_dict can still be changed, specifically properties.\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0;32m---> 73\u001b[0;31m     executor.Do(\n\u001b[0m\u001b[1;32m     74\u001b[0m         copy.deepcopy(input_dict), output_dict, copy.deepcopy(exec_properties))\n\n\u001b[0;32m~/.pyenv/versions/3.8.2/envs/tfx-3.8.2/lib/python3.8/site-packages/tfx/components/statistics_gen/executor.py\u001b[0m in \u001b[0;36mDo\u001b[0;34m(self, input_dict, output_dict, exec_properties)\u001b[0m\n\u001b[1;32m    154\u001b[0m           (split, tfxio_factory(io_utils.all_files_pattern(uri))))\n\u001b[1;32m    155\u001b[0m     \u001b[0;32mif\u001b[0m \u001b[0;32mnot\u001b[0m \u001b[0msplit_and_tfxio\u001b[0m\u001b[0;34m:\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0;32m--> 156\u001b[0;31m       \u001b[0;32mraise\u001b[0m \u001b[0mValueError\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0;34m'No splits for examples artifact: %s'\u001b[0m \u001b[0;34m%\u001b[0m \u001b[0mexamples\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0m\u001b[1;32m    157\u001b[0m     \u001b[0;32mwith\u001b[0m \u001b[0mself\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0m_make_beam_pipeline\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0;34m)\u001b[0m \u001b[0;32mas\u001b[0m \u001b[0mp\u001b[0m\u001b[0;34m:\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m    158\u001b[0m       \u001b[0;32mfor\u001b[0m \u001b[0msplit\u001b[0m\u001b[0;34m,\u001b[0m \u001b[0mtfxio\u001b[0m \u001b[0;32min\u001b[0m \u001b[0msplit_and_tfxio\u001b[0m\u001b[0;34m:\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\n\u001b[0;31mValueError\u001b[0m: No splits for examples artifact: Artifact(artifact: id: 1\ntype_id: 14\nuri: \"./examples_directory_build/\"\ncustom_properties {\n  key: \"is_external\"\n  value {\n    int_value: 1\n  }\n}\ncustom_properties {\n  key: \"tfx_version\"\n  value {\n    string_value: \"1.12.0\"\n  }\n}\n, artifact_type: id: 14\nname: \"Examples\"\nproperties {\n  key: \"span\"\n  value: INT\n}\nproperties {\n  key: \"split_names\"\n  value: STRING\n}\nproperties {\n  key: \"version\"\n  value: INT\n}\nbase_type: DATASET\n)"
}

ansi_escape = re.compile(r'\x1B(?:[@-Z\\-_]|\[[0-?]*[ -/]*[@-~])')
result = ansi_escape.sub('', dict_output["stack"])

print(result)